In [1]:
import numpy as np
import pickle
import pandas as pd
import datetime
import torch

In [12]:
# path = ".\\numerical_results\\ARCH\\lag=3\\seed=0\\x_real_test.torch"
path = ".\\numerical_results\\STOCKS\\SPX\\seed=0\\x_real.torch"

In [ ]:
with open(path, "rb") as f:
    data = pickle.load(f)

data

In [15]:
data.mean(), data.std()

(tensor(1.9749e-06, device='cuda:0'), tensor(0.9996, device='cuda:0'))

In [16]:
data.shape

torch.Size([4634, 6, 2])

In [21]:
oxford = pd.read_csv(".\data\oxfordmanrealizedvolatilityindices.csv")

In [22]:
start = '2005-01-01 00:00:00+01:00'
end = '2020-01-01 00:00:00+01:00'

df_spx = oxford[oxford['Symbol'] == '.SPX'].set_index(['Unnamed: 0'])[start:end]
df_dji = oxford[oxford['Symbol'] == '.DJI'].set_index(['Unnamed: 0'])[start:end]
index = df_dji.index.intersection(df_spx.index)
df_dji = df_dji.loc[index]
df_spx = df_spx.loc[index]
price_spx = np.log(df_spx[['close_price']].values)
rtn_spx = (price_spx[1:] - price_spx[:-1]).reshape(1, -1, 1)
vol_spx = np.log(df_spx[['medrv']].values).reshape(1, -1, 1)
price_dji = np.log(df_dji[['close_price']].values)
rtn_dji = (price_dji[1:] - price_dji[:-1]).reshape(1, -1, 1)
vol_dji = np.log(df_dji[['medrv']].values).reshape(1, -1, 1)
data_raw = np.concatenate([rtn_spx, vol_spx[:, 1:], rtn_dji, vol_dji[:, 1:]], axis=-1)

In [30]:
data_raw.shape

(1, 3389, 4)

In [ ]:
data_raw

In [ ]:
df_spx

In [ ]:
datasets = []
start = datetime.datetime( 1970,  1,  1,  0,  0, 0 )
end   = datetime.datetime( 2023, 12, 31, 23, 59, 0 )

assets = ("BTC","ETH")

def transfer_percentage_seq(x):
    start = x[:, 0:1, :]
    # remove zero start
    idx_ = torch.nonzero(start == 0, as_tuple=False).tolist()
    if idx_:
        idx_ = idx_[0]
    idx_ = list(set(list(range(x.shape[0]))) - set(idx_))

    new_x = x[idx_, ...]
    new_start = start[idx_, ...]

    new_x = (new_x - new_start) / new_start
    return new_x

def rolling_window(x, x_lag, add_batch_dim=True):
    if add_batch_dim:
        x = x[None, ...]
    return torch.cat([x[:, t:t + x_lag] for t in range(x.shape[1] - x_lag)], dim=0)

for asset in assets:
    data = pd.read_csv(f'.\\data\\binance\\{asset}USDT_1h.csv').set_index('Unnamed: 0')
    data['timestamp'] = pd.to_datetime(data['timestamp'], format='%Y-%m-%d %H:%M:%S') 
    data = data.set_index(['timestamp'])

    if data.index[0] > start:
        start = data.index[0]
    if data.index[-1] < end:
        end = data.index[-1]

    data = data[start:end][["close"]].to_numpy(dtype='float')
    
    data = torch.FloatTensor(data)
    data = rolling_window(data, 5, add_batch_dim=True)
    data = transfer_percentage_seq(data)

    print(data)

    datasets.append(data)
    print(f'\tRolled data for training, shape {list(data.shape)}')
    # print('\tExample : {:.6f}, {:.6f}, {:.6f} ...'.format(*[i.item() for i in dataset[0][:3]]))
    print()

In [ ]:
torch.cat(datasets, dim=1)

In [8]:
datasets

[tensor([[28380.5996],
         [28571.9707],
         [28872.2500],
         ...,
         [29470.8203],
         [29544.0195],
         [29550.0000]]),
 tensor([[ 727.3000],
         [ 737.2000],
         [ 739.8700],
         ...,
         [1879.2100],
         [1884.0000],
         [1883.0699]])]

In [ ]:
assets = ("BTC","ETH")
datasets = []

def rolling_window(x: torch.Tensor, n_lags: int):
    """
    ex
    """
    # https://www.geeksforgeeks.org/python-pytorch-stack-method/
    return torch.stack([x[t:t + n_lags, :] for t in range(x.shape[0] - n_lags + 1)], dim=0)

def transfer_percentage_seq(x):
    start = x[:, 0 :1, :]
    # remove zero start
    idx_ = torch.nonzero(start == 0, as_tuple=False).tolist()
    if idx_:
        idx_ = idx_[0]
    idx_ = list(set(list(range(x.shape[0]))) - set(idx_))

    new_x = x[idx_, ...]
    new_start = start[idx_, ...]

    new_x = (new_x - new_start) / new_start
    return new_x

for asset in assets:
    df = pd.read_csv(f'.\\data\\binance\\{asset}USDT_1h.csv')
    # print(df)
    print(f'Preprocess data: {asset}, shape {df.shape}')
    dataset = df[['close']].to_numpy(dtype='float')
    print(dataset)

    dataset = torch.FloatTensor(dataset)
    dataset = rolling_window(dataset, 5)
    # dataset = transfer_percentage_seq(dataset)

    datasets.append(dataset)

    print(dataset)
    print(f'\tRolled data for training, shape {list(dataset.shape)}')
    # print('\tExample : {:.6f}, {:.6f}, {:.6f} ...'.format(*[i.item() for i in dataset[0][:3]]))
    print()

print("----------------------")
print(datasets)


In [20]:
experiment_x = torch.Tensor( [[[28380.5996,   727.3000],
         [28571.9707,   737.2000],
         [28872.2500,   739.8700],
         [28897.8301,   738.8300],
         [29126.6992,   743.9600],
         [28571.9707,   737.2000],
         [28872.2500,   739.8700],
         [28897.8301,   738.8300],
         [29126.6992,   743.9600],
         [28966.3594,   738.2400],
         [28872.2500,   739.8700],
         [28897.8301,   738.8300],
         [29126.6992,   743.9600],
         [28966.3594,   738.2400],
         [29100.8398,   740.4800]]] )

In [3]:
experiment_x.shape

torch.Size([1, 15, 2])

In [54]:
def transfer_percentage_seq(x):
    start = x[:, 0:1, :]
    # remove zero start
    idx_ = torch.nonzero(start == 0, as_tuple=False).tolist()
    if idx_:
        idx_ = idx_[0]
    idx_ = list(set(list(range(x.shape[0]))) - set(idx_))

    new_x = x[idx_, ...]
    new_start = start[idx_, ...]

    new_x = (new_x - new_start) / new_start
    return new_x

def rolling_window(x, p_lag, q_lag, add_batch_dim=True):
    window = p_lag + q_lag
    print(window, x.shape, x.shape[1]-window)
    
    # if add_batch_dim:
    #     x = x[None, ...]

    combined_segments = []
    for t in range(x.shape[1]-window):
        print(t)
        # p_segment = x[:,:, t      :t+p_lag]
        # q_segment = x[:,:, t+p_lag:t+window]
        p_segment = x[:, t      :t+p_lag]
        p_segment = transfer_percentage_seq(p_segment)
        q_segment = x[:, t+p_lag:t+window]
        q_segment = transfer_percentage_seq(q_segment)
        combined_segment = torch.cat([p_segment, q_segment], dim=1)
        combined_segments.append(combined_segment)
        print( torch.cat([p_segment, q_segment], dim=1) )
    
    return torch.cat(combined_segments, dim=0)

In [56]:
res = rolling_window(experiment_x, 2,4)
res.shape

6 torch.Size([1, 15, 2]) 9
0
tensor([[[ 0.0000,  0.0000],
         [ 0.0067,  0.0136],
         [ 0.0000,  0.0000],
         [ 0.0009, -0.0014],
         [ 0.0088,  0.0055],
         [-0.0104, -0.0036]]])
1
tensor([[[ 0.0000,  0.0000],
         [ 0.0105,  0.0036],
         [ 0.0000,  0.0000],
         [ 0.0079,  0.0069],
         [-0.0113, -0.0022],
         [-0.0009,  0.0014]]])
2
tensor([[[ 0.0000,  0.0000],
         [ 0.0009, -0.0014],
         [ 0.0000,  0.0000],
         [-0.0190, -0.0091],
         [-0.0087, -0.0055],
         [-0.0079, -0.0069]]])
3
tensor([[[0.0000, 0.0000],
         [0.0079, 0.0069],
         [0.0000, 0.0000],
         [0.0105, 0.0036],
         [0.0114, 0.0022],
         [0.0194, 0.0092]]])
4
tensor([[[ 0.0000,  0.0000],
         [-0.0190, -0.0091],
         [ 0.0000,  0.0000],
         [ 0.0009, -0.0014],
         [ 0.0088,  0.0055],
         [ 0.0033, -0.0022]]])
5
tensor([[[ 0.0000,  0.0000],
         [ 0.0105,  0.0036],
         [ 0.0000,  0.0000],
      

torch.Size([9, 6, 2])

In [12]:
torch.cat(datasets, dim=-1)

tensor([[[28380.5996,   727.3000],
         [28571.9707,   737.2000],
         [28872.2500,   739.8700],
         [28897.8301,   738.8300],
         [29126.6992,   743.9600]],

        [[28571.9707,   737.2000],
         [28872.2500,   739.8700],
         [28897.8301,   738.8300],
         [29126.6992,   743.9600],
         [28966.3594,   738.2400]],

        [[28872.2500,   739.8700],
         [28897.8301,   738.8300],
         [29126.6992,   743.9600],
         [28966.3594,   738.2400],
         [29100.8398,   740.4800]],

        ...,

        [[29488.3594,  1880.2000],
         [29460.0000,  1878.1899],
         [29384.0098,  1873.8000],
         [29412.1699,  1871.8800],
         [29470.8203,  1879.2100]],

        [[29460.0000,  1878.1899],
         [29384.0098,  1873.8000],
         [29412.1699,  1871.8800],
         [29470.8203,  1879.2100],
         [29544.0195,  1884.0000]],

        [[29384.0098,  1873.8000],
         [29412.1699,  1871.8800],
         [29470.8203,  1879.210

In [3]:
matrix = [
    [
        [1,2,3],
        [4,5,6]
    ],[
        [7,8,9],
        [10,11,12]
    ]
]
matrix = torch.Tensor(matrix)

In [5]:
matrix[..., 2]

tensor([[ 3.,  6.],
        [ 9., 12.]])

In [7]:
matrix[..., [1,2] ]

tensor([[[ 2.,  3.],
         [ 5.,  6.]],

        [[ 8.,  9.],
         [11., 12.]]])